In [41]:
from arcgis.gis import GIS
import json
import pandas as pd
import urllib.request as request
tokens = pd.read_csv(r"C:\Users\M3ECHJJJ\Documents\Tokens.csv", index_col='Site')

In [43]:
gis = GIS(tokens.at['uCOP','URL']))
br_layer_item = gis.content.get('f74235ef42c547e3ad976135d0761c98')
br_layers = br_layer_item.layers
df=br_layers[0].query(where="flagroe = 'Valid' OR flagroe = 'Geocode Error'").sdf
#br_features = br_fset.features
br_flayer = br_layers[0]

In [17]:
def geocode(addressLine):
    addressLine = addressLine.replace(' ','%20')
    with request.urlopen(r'https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates?Address=' + addressLine + '&outFields=*&forStorage=false&f=pjson') as response:
        source = response.read()
        data = json.loads(source)
    y1 = data['candidates'][0].get("location",{}).get("y")
    x1 = data['candidates'][0].get("location",{}).get("x")
    #number1 = data['candidates'][0].get("attributes",{}).get("AddNum")
    #street1 = data['candidates'][0].get("attributes",{}).get("StName")
    #city1 = data['candidates'][0].get("attributes",{}).get("City")
    #state1 = data['candidates'][0].get("attributes",{}).get("Subregion")
    #zip1 = data['candidates'][0].get("attributes",{}).get("Postal")
    score1 = int(data['candidates'][0].get("score",{}))
        
    return x1, y1, score1

In [18]:
null_list = []

In [19]:
for index,row in df.iterrows():
    #print(f.geometry)
    try:
        if row['SHAPE']['x']>-80 or row['SHAPE']['x']<-85 or row['SHAPE']['y']>28 or row['SHAPE']['y']<24:
            null_list.append(row['OBJECTID'])
    except:
        null_list.append(row['OBJECTID'])

In [186]:
updates_to_push = []

In [189]:
i=1
for index,row in df[df.OBJECTID.isin(null_list)].iterrows():
    try:
        print('{0}/{1}'.format(i,str(len(null_list))), end = "\r")
        addressLine = row['appaddress']+' '+row['appcityname'] +' '+row['appstate']+''+str(row['appzipcode'])+row['appcountyname']
        x, y, score = geocode(addressLine)
        if score >=90:
            feat = {"geometry":{"x":x,"y":y},"attributes":{'OBJECTID':row['OBJECTID'],'latitude':y,'longitude':x,'flagroe':'Valid'}}
        else:
            feat = {'geometry':{'x':float(x),'y':float(y)},'attributes':{'OBJECTID':row['OBJECTID'],'latitude':float(y),'longitude':float(x),'flagroe':'Geocode Error'}}
        updates_to_push.append(feat)
        i=i+1
    except Exception as e:
        print(e)

'ascii' codec can't encode character '\u2019' in position 96: ordinal not in range(128)
HTTP Error 403: 
HTTP Error 403: 
HTTP Error 403: 
HTTP Error 403: 
'ascii' codec can't encode character '\u0142' in position 106: ordinal not in range(128)
HTTP Error 403: 
'ascii' codec can't encode character '\xe7' in position 105: ordinal not in range(128)
list index out of range
'ascii' codec can't encode character '\xe9' in position 122: ordinal not in range(128)
HTTP Error 403: 
'ascii' codec can't encode character '\xe1' in position 97: ordinal not in range(128)
'ascii' codec can't encode character '\xe3' in position 113: ordinal not in range(128)


In [190]:
update_result = br_flayer.edit_features(updates=updates_to_push, rollback_on_failure=False)

[315170, 315207, 315236, 315245, 315266, 315309, 315318, 315321, 315332, 315333, 315335, 315365, 315392, 315395, 315444, 315458, 315465, 315466, 315836]

In [20]:
len(null_list)

471